# ETL da camada bronze para camada silver

Este notebook realiza o ETL dos dados da camada bronze para a camada silver. Ou seja: ele abre o dataset e o salva num dataframe, realiza a transformação dos dados e os carrega num arquivo `.csv` e no banco de dados.

## EXTRACT
# Estratégia de Leitura Escalável (PySpark)

Para contornar as limitações de memória (especialmente em ambientes como WSL) ao ler o arquivo `MICRODADOS_ENEM_2021.csv` (>1.5GB), adotamos a estratégia de processamento distribuído com **Apache Spark**.

Diferente do padrão do Pandas que tenta carregar todo o arquivo na RAM de uma vez, o PySpark utiliza o conceito de *Lazy Evaluation* (avaliação preguiçosa). Ele mapeia o arquivo e cria um plano de execução, mas só processa os dados na memória quando uma ação é solicitada, evitando o travamento do sistema.

#### **Parâmetros Críticos:**

* **`inferSchema="true"`:** Permite que o Spark percorra os dados inicialmente para identificar automaticamente quais colunas são numéricas e quais são textuais, facilitando a análise imediata.
* **`encoding="ISO-8859-1"`:** Corrige erros de decodificação de caracteres que contem dentro do arquivo `MICRODADOS_ENEM_2021.csv` que e um "erro" bem comuns em dados  usado no Brasil.
* **`delimiter=";"`:** Define o ponto e vírgula como o separador correto, evitando que o dataset seja interpretado equivocadamente como uma única coluna longa.

In [ ]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("RawToSilves") \
    .config("spark.driver.memory", "5g") \
    .getOrCreate()

data_layer_filepath = '../raw/'

df = spark.read \
    .option("header", "true") \
    .option("delimiter", ";") \
    .option("encoding", "ISO-8859-1") \
    .option("inferSchema", "true") \
    .csv(data_layer_filepath + 'data_raw/MICRODADOS_ENEM_2021.csv')

print("Arquivo completo mapeado com sucesso!")

df.show(5)

##df.printSchema()

## TRANSFORM
### Padronização dos Nomes das Colunas.

Inicialmente, o dataset contém colunas com nomes sem padrão, por "sorte" nessa basse que estamos usando todos os nome estão padronizado porem tudo com letras Maisculas. E para fins de tratamento iremos fazer uma varedura para que caso ajá alguma coluna que não eteja separam palavras com `_`, ou tenha alguma coluna com `whitespace`, passse para o padrão com as colunas seguindo esse novo padrão: **todos os caracteres em minúsculo, separando palavras com `_`**.


Para renomear todas as colunas de uma vez convertendo para minúsculas e trocando espaços por `_`, a maneira mais eficiente quando se usa o **PySpark** é usar o método `.toDF().`

In [ ]:

novas_colunas = [col.lower().replace(' ', '_') for col in df.columns]

df = df.toDF(*novas_colunas)

print(df.columns)
df.show(5)

### Remoção de Colunas Desnecessárias

Afim de melhorar como os dados vão ficar nessa etapa começamos a filtrar e retirar alguns dados são desnecesarios como por exemplo **nu_ano** já que toda a base e referente ao ano de 2021, e assim optamos por não trabalhar com essa coluna.

In [ ]:
cols_to_drop = ['nu_ano']

df = df.drop(*cols_to_drop) 

for col in cols_to_drop:
    if col not in df.columns:
        print(f"Coluna {col} deletada!")

print("Colunas restantes: ")
print(df.columns)
print(" ")
print("Tabelas restantes: ")
df.show(5)

In [ ]:
import pandas as pd
import os
from psycopg import connect, sql
from dotenv import load_dotenv
import sys


DB_SCHEMA = "silver" 
TABLE_NAME = "dados_inep" 
TABLE_FULL_NAME = f"{DB_SCHEMA}.{TABLE_NAME}"

print("--- Iniciando processo de carga e verificação no PostgreSQL ---")


try:

    ddl_path = data_layer_filepath + 'silver/silver_ddl.sql' 
    ddl = open(ddl_path).read().replace('\n', ' ')
except Exception as e:
    print(f'Erro ao abrir arquivo ddl: {e}')
    ddl = None 

def get_db_connection_info():
    load_dotenv()
    
    url = os.getenv('DB_URL')
    db_env = os.getenv('DB_ENV')
    if url is not None and db_env == 'prod':
        return url

   
    DB_USER = "admin"           
    DB_PASSWORD = "l1l2r1r2"    
    DB_HOST = "localhost"       
    DB_PORT = "5432"
    DB_NAME = "dados_inep"

    conn_string = f"host={DB_HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} port={DB_PORT}"
    return conn_string

conn_info = get_db_connection_info()


print(f"Total de linhas a serem carregadas: {len(df)}")

cols = list(df.columns)

insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
    sql.Identifier(DB_SCHEMA, TABLE_NAME),                      
    sql.SQL(", ").join(map(sql.Identifier, cols)),               
    sql.SQL(", ").join(sql.Placeholder() * len(cols))            
)

def get_number_of_rows(cur):
    
    try:
        query = sql.SQL('select count(*) from {};').format(sql.Identifier(DB_SCHEMA, TABLE_NAME))
        cur.execute(query)
        return cur.fetchone()[0]
    except:
        return 0

with connect(conn_info) as conn:
    print("\nConexão com o PostgreSQL (dados_inep) estabelecida.")
    
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE SCHEMA IF NOT EXISTS {};").format(sql.Identifier(DB_SCHEMA)))
        
        if ddl:
            print("Aplicando DDL...")
            cur.execute(ddl)
            conn.commit()
            print("Estrutura do banco de dados verificada.")
        
        print(f'Existem {get_number_of_rows(cur)} linhas na tabela {TABLE_FULL_NAME}!')
        print("Iniciando carga de dados...")

        
        counter = 0
        for _, row in df.iterrows():
            values = [None if pd.isna(v) else v for v in row]
            cur.execute(insert_query, values)
            
            counter += 1
            if counter % 10000 == 0:
                conn.commit()
                print(f"{counter} linhas processadas...")

        conn.commit()
        print("Carga concluída com sucesso!")
        print(f'Total final: {get_number_of_rows(cur)} linhas no banco de dados!')